In [1]:
import pandas as pd

# Cargar el conjunto de datos
data_path = './CTG.csv'
data = pd.read_csv(data_path)

# Mostrar las primeras filas del conjunto de datos y un resumen de los mismos
data.head(), data.describe(), data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2129 entries, 0 to 2128
Data columns (total 40 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   FileName  2126 non-null   object 
 1   Date      2126 non-null   object 
 2   SegFile   2126 non-null   object 
 3   b         2126 non-null   float64
 4   e         2126 non-null   float64
 5   LBE       2126 non-null   float64
 6   LB        2126 non-null   float64
 7   AC        2126 non-null   float64
 8   FM        2127 non-null   float64
 9   UC        2127 non-null   float64
 10  ASTV      2127 non-null   float64
 11  MSTV      2127 non-null   float64
 12  ALTV      2127 non-null   float64
 13  MLTV      2127 non-null   float64
 14  DL        2128 non-null   float64
 15  DS        2128 non-null   float64
 16  DP        2128 non-null   float64
 17  DR        2128 non-null   float64
 18  Width     2126 non-null   float64
 19  Min       2126 non-null   float64
 20  Max       2126 non-null   floa

(       FileName       Date      SegFile      b       e    LBE     LB   AC  \
 0  Variab10.txt  12/1/1996  CTG0001.txt  240.0   357.0  120.0  120.0  0.0   
 1    Fmcs_1.txt   5/3/1996  CTG0002.txt    5.0   632.0  132.0  132.0  4.0   
 2    Fmcs_1.txt   5/3/1996  CTG0003.txt  177.0   779.0  133.0  133.0  2.0   
 3    Fmcs_1.txt   5/3/1996  CTG0004.txt  411.0  1192.0  134.0  134.0  2.0   
 4    Fmcs_1.txt   5/3/1996  CTG0005.txt  533.0  1147.0  132.0  132.0  4.0   
 
     FM   UC  ...    C    D    E   AD   DE   LD   FS  SUSP  CLASS  NSP  
 0  0.0  0.0  ...  0.0  0.0  0.0  0.0  0.0  0.0  1.0   0.0    9.0  2.0  
 1  0.0  4.0  ...  0.0  0.0  0.0  1.0  0.0  0.0  0.0   0.0    6.0  1.0  
 2  0.0  5.0  ...  0.0  0.0  0.0  1.0  0.0  0.0  0.0   0.0    6.0  1.0  
 3  0.0  6.0  ...  0.0  0.0  0.0  1.0  0.0  0.0  0.0   0.0    6.0  1.0  
 4  0.0  5.0  ...  0.0  0.0  0.0  0.0  0.0  0.0  0.0   0.0    2.0  1.0  
 
 [5 rows x 40 columns],
                  b            e          LBE           LB        

In [2]:
# Análisis detallado de variables categóricas
categorical_columns = ['FileName', 'Date', 'SegFile']
categorical_data = data[categorical_columns].describe(include='all')

# Frecuencias de las categorías en cada variable categórica
category_frequencies = {col: data[col].value_counts() for col in categorical_columns}

# Matriz de correlación de las variables numéricas
correlation_matrix = data.drop(columns=categorical_columns).corr()

categorical_data, category_frequencies, correlation_matrix

(            FileName       Date      SegFile
 count           2126       2126         2126
 unique           352         48         2126
 top     S8001034.dsp  2/22/1995  CTG0001.txt
 freq              34        240            1,
 {'FileName': FileName
  S8001034.dsp    34
  S7001029.dsp    33
  S8001037.dsp    30
  S8001038.dsp    26
  S7001027.dsp    24
                  ..
  S7001006.dsp     1
  S7001008.dsp     1
  Mepfp_3.txt      1
  Mrmmf_1.txt      1
  Variab10.txt     1
  Name: count, Length: 352, dtype: int64,
  'Date': Date
  2/22/1995     240
  5/2/1996      160
  7/18/1996     101
  10/3/1996      92
  5/3/1996       88
  5/10/1998      85
  5/1/1996       79
  10/6/1996      79
  6/6/1998       77
  9/25/1996      74
  1/8/1995       71
  10/25/1997     70
  1/4/1980       56
  8/20/1996      55
  11/26/1997     53
  4/8/1998       53
  5/6/1996       53
  8/28/1996      52
  12/9/1996      50
  8/16/1996      43
  7/16/1996      42
  11/14/1997     38
  9/23/1996      3

In [3]:
# Identificar columnas con valores faltantes
missing_data = data.isnull().sum()
missing_data = missing_data[missing_data > 0]

# Opciones de imputación para valores faltantes
# Calcular media, mediana y moda para las columnas numéricas con valores faltantes
imputation_values = {}
for column in missing_data.index:
    if data[column].dtype != 'O':  # 'O' significa objeto, típicamente strings para pandas
        mean = data[column].mean()
        median = data[column].median()
        mode = data[column].mode()[0]
        imputation_values[column] = {'mean': mean, 'median': median, 'mode': mode}

missing_data, imputation_values

(FileName    3
 Date        3
 SegFile     3
 b           3
 e           3
 LBE         3
 LB          3
 AC          3
 FM          2
 UC          2
 ASTV        2
 MSTV        2
 ALTV        2
 MLTV        2
 DL          1
 DS          1
 DP          1
 DR          1
 Width       3
 Min         3
 Max         3
 Nmax        3
 Nzeros      3
 Mode        3
 Mean        3
 Median      3
 Variance    3
 Tendency    3
 A           3
 B           3
 C           3
 D           3
 E           3
 AD          3
 DE          3
 LD          3
 FS          3
 SUSP        3
 CLASS       3
 NSP         3
 dtype: int64,
 {'b': {'mean': 878.4397930385701, 'median': 538.0, 'mode': 0.0},
  'e': {'mean': 1702.8772342427094, 'median': 1241.0, 'mode': 1199.0},
  'LBE': {'mean': 133.3038570084666, 'median': 133.0, 'mode': 133.0},
  'LB': {'mean': 133.3038570084666, 'median': 133.0, 'mode': 133.0},
  'AC': {'mean': 2.722483537158984, 'median': 1.0, 'mode': 0.0},
  'FM': {'mean': 7.503055947343676, 'median'